## imports

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..'))  # Append parent directory to Python path

import pandas as pd
from scripts.data_loader import load_data
from scripts.plot_util import PlotHelper
# from scripts.preprocess import preprocess_data



load data basic eda

In [ ]:
df = load_data("../data/raw_analyst_ratings.csv")
print(df.head())
print(df.columns)

In [ ]:
print(df.info())
print(df.shape)

In [ ]:
print(type(df['headline']))
print(df['headline'].head())

In [ ]:
# --- 1. Handling Null Values ---

print("\n--- 1. Null Value Check (Before Cleaning) ---")
print(df.isnull().sum())

# We focus on the 'headline' column since it's used for sentiment analysis.
# Strategy: Drop rows where the 'headline' is null, as we can't analyze empty text.
df.dropna(subset=['headline'], inplace=True)
print("\n--- Null Value Check (After Cleaning) ---")
print(f"Remaining rows after dropping null headlines: {df.shape[0]}")

In [ ]:
# --- 2. Date Structure and Time Series Preparation  ---

if 'date' in df.columns:
    
    # 1. Force conversion using 'ISO8601' format and ensure UTC conversion.
    # Setting utc=True handles the mixed timezone offsets by normalizing all values to UTC 
    # and ensures the resulting column is a proper datetime dtype.
    df['date'] = pd.to_datetime(df['date'], format='ISO8601', utc=True)
    
    print("\n--- 2. Date Column Structure (After Conversion) ---")
    print(df['date'].head())
    print(f"Data type is now: {df['date'].dtype}")
    # 2. Set the Index
    # Note: Since utc=True was used, the dates are already in UTC.
    df.set_index('date', inplace=True)
    print("\nDataFrame Index is now set to Date (UTC).")
    print(df.head()) 
    
else:
    print("\nWarning: Date column not found. Skipping date conversion.")

In [ ]:
# --- 3. Duplicate and Consistency Checks ---

# Check for duplicate rows across all columns
duplicate_count = df.duplicated().sum()
print(f"\nTotal duplicate rows found: {duplicate_count}")

if duplicate_count > 0:
    # Drop duplicates, keeping the first instance
    df.drop_duplicates(inplace=True)
    print(f"Rows remaining after dropping duplicates: {df.shape[0]}")

# Ensure the 'headline' column is string type before analysis
df['headline'] = df['headline'].astype(str)

# Descriptive Statistics

In [ ]:
# Calculate headline lengths
df['headline_length'] = df['headline'].str.len()

# Basic statistics
headline_stats = df['headline_length'].describe()
print("Headline Length Statistics:")
print(headline_stats)

# Visualize with PlotHelper
helper = PlotHelper()
helper.histogram(df, column='headline_length', bins=20, title='Distribution of Headline Lengths')

In [ ]:
# Count articles per publisher
publisher_counts = df['publisher'].value_counts()
print("Number of Articles per Publisher:")
print(publisher_counts)

# Visualize top 10 with PlotHelper
top_publishers = publisher_counts.head(10).reset_index()
top_publishers.columns = ['publisher', 'count']
helper.bar(top_publishers, x='publisher', height='count', title='Top 10 Most Active Publishers')

In [ ]:
# Ensure 'date' is datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Group by date and count articles
daily_counts = df.groupby(df['date'].dt.date).size().reset_index(name='count')
daily_counts['date'] = pd.to_datetime(daily_counts['date'])
print("Daily Article Counts:")
print(daily_counts.head())

# Plot trends over time with PlotHelper
helper.line_plot(daily_counts, x='date', y='count', title='Article Frequency Over Time')

# Analyze by day of week
df['day_of_week'] = df['date'].dt.day_name()
weekly_counts = df['day_of_week'].value_counts().reset_index()
weekly_counts.columns = ['day', 'count']
print("Articles by Day of Week:")
print(weekly_counts)

# Visualize days with PlotHelper
helper.bar(weekly_counts, x='day', y='count', title='Articles by Day of the Week')

Text Analysis(Topic Modeling):

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter
import re

# Download NLTK resources if not already done
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Preprocess text function
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())  # Remove punctuation and lowercase
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 2]
    return ' '.join(tokens)

# Apply preprocessing to headlines
df['processed_headline'] = df['headline'].apply(preprocess_text)

# 1. Extract common keywords using TF-IDF
vectorizer = TfidfVectorizer(max_features=100, ngram_range=(1, 2))  # Unigrams and bigrams
tfidf_matrix = vectorizer.fit_transform(df['processed_headline'])
feature_names = vectorizer.get_feature_names_out()
tfidf_scores = tfidf_matrix.sum(axis=0).A1
keywords = sorted(zip(feature_names, tfidf_scores), key=lambda x: x[1], reverse=True)[:20]
print("Top 20 Keywords/Phrases by TF-IDF:")
for word, score in keywords:
    print(f"{word}: {score:.4f}")

# 2. Topic Modeling with LDA
lda = LatentDirichletAllocation(n_components=5, random_state=42)  # 5 topics
lda.fit(tfidf_matrix)

# Display topics
print("\nTop Topics (LDA):")
for topic_idx, topic in enumerate(lda.components_):
    top_words = [feature_names[i] for i in topic.argsort()[:-11:-1]]
    print(f"Topic {topic_idx + 1}: {', '.join(top_words)}")

# 3. Identify significant events/phrases (e.g., "FDA approval", "price target")
significant_phrases = ['fda approval', 'price target', 'earnings report', 'merger', 'acquisition', 'stock split', 'dividend']
phrase_counts = Counter()
for headline in df['processed_headline']:
    for phrase in significant_phrases:
        if phrase in headline:
            phrase_counts[phrase] += 1

print("\nCounts of Significant Phrases:")
for phrase, count in phrase_counts.most_common():
    print(f"{phrase}: {count}")

Time Series Analysis:

How does the publication frequency vary over time? Are there spikes in article publications related to specific market events?
Analysis of publishing times might reveal if there’s a specific time when most news is released, which could be crucial for traders and automated trading systems.


In [ ]:

# 1. Publication Frequency Over Time
# Resample to daily frequency and count articles
daily_freq = df.resample('D').size()
print("Daily Publication Frequency:")
print(daily_freq.head(10))

# Plot daily frequency
helper.line_plot(daily_freq.reset_index(), x='date', y=0, title='Daily Publication Frequency Over Time')

# 2. Identify Spikes (e.g., days with significantly higher publications)
# Calculate rolling mean and std for anomaly detection
rolling_mean = daily_freq.rolling(window=7).mean()
rolling_std = daily_freq.rolling(window=7).std()
threshold = rolling_mean + 2 * rolling_std
spikes = daily_freq[daily_freq > threshold]
print("\nPotential Spike Days (above 2 std from 7-day rolling mean):")
print(spikes.head(10))

# Note: To relate to market events, you would need external data (e.g., stock prices, earnings dates).
# For now, this identifies statistical spikes. Manually check headlines on spike dates for events.

# 3. Analysis of Publishing Times
# Extract hour of day from index
df['hour'] = df.index.hour
hourly_counts = df.groupby('hour').size()
print("\nPublication Counts by Hour of Day:")
print(hourly_counts)

# Plot hourly distribution
helper.bar(hourly_counts.reset_index(), x='hour', y=0, title='Publications by Hour of Day')


Publisher Analysis:

- Which publishers contribute most to the news feed? Is there a difference in the type of news they report?

- If email addresses are used as publisher names, identify unique domains to see if certain organizations contribute more frequently.


In [ ]:
# Publisher Analysis

# 1. Which publishers contribute most to the news feed?
publisher_counts = df['publisher'].value_counts()
print("Top 10 Publishers by Number of Articles:")
print(publisher_counts.head(10))

# To see if there's a difference in the type of news they report,
# let's look at the most common words in headlines for the top 3 publishers
from collections import Counter
import re

def get_common_words(headlines, top_n=10):
    all_words = []
    for headline in headlines:
        words = re.findall(r'\b\w+\b', headline.lower())
        all_words.extend(words)
    # Remove common stop words (basic list)
    stop_words = set(['the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'are', 'was', 'were', 'be', 'been', 'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could', 'should', 'may', 'might', 'must', 'can', 'this', 'that', 'these', 'those'])
    filtered_words = [word for word in all_words if word not in stop_words and len(word) > 2]
    return Counter(filtered_words).most_common(top_n)

top_publishers = publisher_counts.head(3).index
for publisher in top_publishers:
    headlines = df[df['publisher'] == publisher]['headline']
    common_words = get_common_words(headlines)
    print(f"\nCommon words for {publisher}:")
    for word, count in common_words:
        print(f"{word}: {count}")

# 2. If email addresses are used as publisher names, identify unique domains
email_publishers = df['publisher'].str.contains('@', na=False)
if email_publishers.any():
    domains = df[email_publishers]['publisher'].str.split('@').str[1].str.split('.').str[0]
    unique_domains = domains.value_counts()
    print("\nUnique domains from email publishers:")
    print(unique_domains)
else:
    print("\nNo email addresses found in publisher names.")